### 文档拆分器 Text Splitters

#### CharacterTextSplitter：Split by character

举例1：纯字符数分割

In [16]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.示例文本
text = """
LangChain 是一个用于开发由语言模型驱动的应用程序的框架的。它提供了一套工具和抽象，使开发者能够更容易地构建复杂的应用程序。
"""

# 3.定义字符分割器
splitter = CharacterTextSplitter(
    chunk_size=50, # 每块大小
    chunk_overlap=5,# 块与块之间的重复字符数
    # length_function=len, # 用于计算切块长度的方法。默认只计算字符数，但通常这里会使用Token。
    separator=""   # 禁用分隔符优先
)

# 4.分割文本
texts = splitter.split_text(text)

# 5.打印结果
for i, chunk in enumerate(texts):
    print(f"块 {i+1}:长度：{len(chunk)}")
    print(chunk)
    print("-" * 50)

块 1:长度：49
LangChain 是一个用于开发由语言模型驱动的应用程序的框架的。它提供了一套工具和抽象，使开发
--------------------------------------------------
块 2:长度：22
象，使开发者能够更容易地构建复杂的应用程序。
--------------------------------------------------


举例2：指定分割符

In [20]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.定义要分割的文本
text = "这是一个示例文本啊。我们将使用CharacterTextSplitter将其分割成小块。分割基于字符数。123。"

# text = """
# LangChain 是一个用于开发由语言模型。驱动的应用程序的框架的。它提供了一套工具和抽象。使开发者能够更容易地构建复杂的应用程序。
# """

# 3.定义分割器实例
text_splitter = CharacterTextSplitter(
    chunk_size=43,   # 每个块的最大字符数
    chunk_overlap=5, # 块之间的重叠字符数
    separator="。",  # 按句号分割
    keep_separator=True
)

# 4.开始分割
chunks = text_splitter.split_text(text)

# 5.打印效果
for  i,chunk in enumerate(chunks):
    print(f"块 {i + 1}:长度：{len(chunk)}")
    print(chunk)
    print("-"*50)


块 1:长度：43
这是一个示例文本啊。我们将使用CharacterTextSplitter将其分割成小块
--------------------------------------------------
块 2:长度：13
。分割基于字符数。123。
--------------------------------------------------


**注意：无重叠。**

**separator优先原则**：当设置了 `separator`（如"。"），分割器会首先尝试在分隔符处分割，然后再考虑 chunk_size。这是为了避免在句子中间硬性切断。这种设计是为了：

1. 优先保持语义完整性（不切断句子）
2. 避免产生无意义的碎片（如半个单词/不完整句子）
3. chunk_overlap仅在合并后的片段之间生效（如果 `chunk_size` 足够大）。如果没有合并的片段，则 overlap失效。
4. 如果 `chunk_size` 比片段小，无法拆分片段，导致 overlap失效。

In [25]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.定义要分割的文本
text = "这是第一段文本。这是第二段内容。最后一段结束。"

# 3.定义字符分割器
text_splitter = CharacterTextSplitter(
    separator="。",
    chunk_size=20,
    chunk_overlap=10,
    keep_separator=True
)

# 4.分割文本
chunks = text_splitter.split_text(text)

# 5.打印结果
for  i,chunk in enumerate(chunks):
    print(f"块 {i + 1}:长度：{len(chunk)}")
    print(chunk)
    print("-"*50)

块 1:长度：15
这是第一段文本。这是第二段内容
--------------------------------------------------
块 2:长度：16
。这是第二段内容。最后一段结束。
--------------------------------------------------


#### RecursiveCharacterTextSplitter：最常用

文档切分器中较常用的是`RecursiveCharacterTextSplitter`，遇到`特定字符`时进行分割。默认情况下，它尝试进行切割的字符包括 `["\n\n", "\n", " ", ""]`。此外，还可以考虑添加，。等分割字符。

具体为：该文本分割器接受一个字符列表作为参数，根据第一个字符进行切块，但如果任何切块太大，则会继续移动到下一个字符，并以此类推。



**特点**：

- **智能分段**：通过递归尝试多种分隔符（如换行符、句号、空格等），将文本分割为大小接近chunk_size的片段，同时避免在句子中间截断。
- **保留上下文**：优先在自然语言边界（如段落、句子结尾）处分割，减少信息碎片化。
- **灵活适配**：适用于多种文本类型（代码、Markdown、普通文本等），是LangChain中最通用的文本分割器。

除此之外，还可以指定的参数包括：

- `length_function`：用于计算切块长度的方法。默认只计算字符数，但通常这里会使用Token。
- `chunk_size`：每个切块的最大token数量（由长度函数测量）。
- `chunk_overlap`：相邻两个切块之间的最大重叠token数量。设置重叠可以使信息不会因为在边界处被切断而丢失。
- `add_start_index`：是否在元数据中包含每个切块在原始文档中的起始位置。

In [30]:
# 1.导入相关依赖
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2.定义RecursiveCharacterTextSplitter分割器对象
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=60,  # 每个块中的最大字符数（非单词或Token）
    chunk_overlap=0, # 邻块之间重叠的字符数。重叠的块可以确保如果重要信息横跨两个块，它不会被错过。一般设为chunk_size的10-20%
    length_function=len,
    add_start_index=True,
)

# 3.定义分割的内容
list=["LangChain框架特性\n\n多模型集成(GPT/Claude)\n记忆管理功能\n链式调用设计。文档分析场景示例：需要处理PDF/Word等格式。"]

# 4.分割器分割
paragraphs = text_splitter.create_documents(list)

for para in paragraphs:
    print(para)
    print('-------')

page_content='LangChain框架特性' metadata={'start_index': 0}
-------
page_content='多模型集成(GPT/Claude)
记忆管理功能
链式调用设计。文档分析场景示例：需要处理PDF/Word等格式。' metadata={'start_index': 15}
-------


In [31]:
# 1.导入相关依赖
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 2.打开.txt文件
with open("asset/load/08-ai.txt", encoding="utf-8") as f:
    state_of_the_union = f.read()

# 3.定义RecursiveCharacterTextSplitter（递归字符分割器）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

# 4.分割文本
texts = text_splitter.create_documents([state_of_the_union])

# 5.打印分割文本
for text in texts:
    print(text)


page_content='人工智能（AI）是什么？'
page_content='人工智能（Artificial'
page_content='Intelligence，简称AI）是指由计算机系统模拟人类智能的技术，使其能够执行通常需要人类认知能力的任务，如学习、推理、决策和语言理解。AI的核心目标是让机器具备感知环境、处理信息并自主行动的'
page_content='让机器具备感知环境、处理信息并自主行动的能力。'
page_content='1. AI的技术基础
AI依赖多种关键技术：

机器学习（ML）：通过算法让计算机从数据中学习规律，无需显式编程。例如，推荐系统通过用户历史行为预测偏好。'
page_content='深度学习：基于神经网络的机器学习分支，擅长处理图像、语音等复杂数据。AlphaGo击败围棋冠军便是典型案例。

自然语言处理（NLP）：使计算机理解、生成人类语言，如ChatGPT的对话能力。'
page_content='2. AI的应用场景
AI已渗透到日常生活和各行各业：

医疗：辅助诊断（如AI分析医学影像）、药物研发加速。

交通：自动驾驶汽车通过传感器和AI算法实现安全导航。'
page_content='金融：欺诈检测、智能投顾（如风险评估模型）。

教育：个性化学习平台根据学生表现调整教学内容。

3. AI的挑战与未来
尽管前景广阔，AI仍面临问题：'
page_content='伦理争议：数据隐私、算法偏见（如招聘AI歧视特定群体）。

就业影响：自动化可能取代部分人工岗位，但也会创造新职业。

技术瓶颈：通用人工智能（AGI）尚未实现，当前AI仅擅长特定任务。'
page_content='未来，AI将与人类协作而非替代：医生借助AI提高诊断效率，教师利用AI定制课程。其发展需平衡技术创新与社会责任，确保技术造福全人类。'


利用PDFLoader加载文档，对文档的内容用递归切割器切割

In [40]:
# 1.导入相关依赖
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2.定义PyPDFLoader加载器
loader = PyPDFLoader("/Users/zhangyf/Documents/简历/大模型v3.0/张一帆_大模型算法工程师.pdf")

# 3.加载和切割文档对象
docs = loader.load()
# print(f"第0页：\n{pages[0].page_content}")

# 4.定义切割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

# 5.对pdf内容进行切割得到文档对象
paragraphs = text_splitter.create_documents([docs[1].page_content])
for para in paragraphs:
    print(para.page_content)
    print('-------')

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)


(LoRA)  随着水泥工业智能化发展，大量生产场景需要更专业的自然语言处理支持，例如工艺文档分析、生产数据解释、设备异常预警等。为此基于Qwen开源模型，进行了针对工业领域的模型微调，增强其在垂直领域中的语义理解与问答能力，为水泥工厂提供更精准的知识服务和决策支持
-------
l 收集工业相关技术文档及案例数据，包括设备手册、工艺流程说明、历史问题记录 l 使用Alpaca数据集并结合Langchain工具对文档进行清洗和格式化处理，将文档转换为模型训练所需的指令数据集 l 基于LLaMA-Factory进行工业场景的QLoRA微调训练，包括设备术语、工艺流程等领域的适配优化 l 执行工业语料的增量微调训练，提升模型对特定工业领域术语和文档的语义理解能力 l
-------
l 基于LLaMA-Factory进行工业场景的QLoRA微调训练，包括设备术语、工艺流程等领域的适配优化 l 执行工业语料的增量微调训练，提升模型对特定工业领域术语和文档的语义理解能力 l 执行微调，并将微调后的模型合并保存，方便本地使用  l 使用 vLLM加载微调好的模型，测试生成效果
-------
l 针对水泥工厂的工艺流程和设备术语，模型生成结果上下文关联性显著提高，满足复杂场景需求 l 采用QLoRA微调技术，显著降低训练和推理资源占用，适配本地硬件环境 l 微调后的模型对垂直领域支持度更好，泛化性提高，回答精准，不易出现幻觉  (RAG)
-------
随着水泥工业智能工厂设备类型和工艺流程的日益复杂，生产和运维过程中积累了大量文档数据，包括操作手册、设备维护记录、技术标准和历史故障数据。这些数据具有动态增长的特性，为了更好地挖掘其价值，开发了一套基于大语言模型（LLM）的知识问答系统。系统结合检索增强生成（RAG）技术，支持对动态更新的工业知识进行高效检索与智能问答，为操作员和工程师提供实时支持。针对水泥工业生产中常见的设备异常、工艺偏差和生
-------
基于大语言模型（LLM）的知识问答系统。系统结合检索增强生成（RAG）技术，支持对动态更新的工业知识进行高效检索与智能问答，为操作员和工程师提供实时支持。针对水泥工业生产中常见的设备异常、工艺偏差和生产故障等固定问题场景，系统通过微调训练本地部署的
-------
LLM 模型，将领域知识嵌入模型，进一步优化模型在

####  Split by tokens(了解)

大语言模型通常都有一个最大的输入Token限制。因此，当我们将文本拆分为块时，除了字符以外，更常用的一种方法计算Token的数量。此外，大语言模型(LLM)通常是以token的数量作为其计量(或收费)的依据，所以采用token分割也有助于我们在使用时更好的控制成本。

**"Split by tokens"**（按Token分割）是指根据文本的`Token数量`（而非字符或单词数）将长文本切分成多个小块。这是为了适配大多数语言模型（如GPT）的输入限制（如4096 Token），确保每个文本块不超过模型的Token上限。

这里提供一个Token与字符转化的可视化工具，有OpenAI提供：https://platform.openai.com/tokenizer

1. **什么是Token？**
   - 对模型而言，Token是文本的最小处理单位。例如：
     - 英文：`"hello"` → 1个Token，`"ChatGPT"` → 2个Token（`"Chat"` + `"GPT"`）。
     - 中文：`"人工智能"` → 可能拆分为2-3个Token（取决于分词器）。
2. **为什么按Token分割？**
   - 语言模型对输入长度的限制是基于Token数（如GPT-4的8k/32k Token上限）。
   - 直接按字符或单词分割可能导致实际Token数超限。



In [ ]:
# 1. 导入相关依赖
from langchain_text_splitters import CharacterTextSplitter
import tiktoken  # 用于计算Token数量


# 2. 定义通过Token切割器
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=32,
    chunk_overlap=0,
    separator="。",  # 指定中文句号为分隔符
    keep_separator=False,  # 保留句号
)
# 3.定义文本
text = "人工智能是一个强大的开发框架。它支持多种语言模型和工具链。今天你吃了吗？我好想吃饭，但是现在不饿 咋办 12345不说话"

# 4. 开始切割
texts = text_splitter.split_text(text)
print(f"分割后的块数: {len(texts)}")

# 5. 初始化tiktoken编码器（用于Token计数）
encoder = tiktoken.get_encoding("cl100k_base")  # 确保与CharacterTextSplitter的encoding_name一致

# 6. 打印每个块的Token数和内容
for i, chunk in enumerate(texts):
    tokens = encoder.encode(chunk)  # 现在encoder已定义
    print(f"块 {i + 1}: {len(tokens)} Token\n内容: {chunk}\n")

分割后的块数: 2
块 1: 32 Token
内容: 人工智能是一个强大的开发框架。它支持多种语言模型和工具链

块 2: 37 Token
内容: 今天你吃了吗？我好想吃饭，但是现在不饿 咋办 12345不说话

